# Improved RIDC(p,k)-AB(4)
- (a,b),    endpoints
- alpha,    i.c.s
- N,        steps
- p,        order of method
- K,        # of intervals
- f,        function

In [2]:
import time
import numpy as np
from scipy.interpolate import lagrange
from scipy.integrate import quadrature
from functools import reduce

In [5]:
dy_dt = lambda t, y : 4*t*np.sqrt(y)
y = lambda t: y0*(1+t**2)**2
y0=1

In [3]:
# define Adam Bashforth four step explicit method
# considering placement of stencil
def corrector(y1, y0, t, h, f):
    e1 = f(t[0], y1[0]) - f(t[0], y0[0])
    e2 = f(t[1], y1[1]) - f(t[1], y0[1])
    e3 = f(t[2], y1[2]) - f(t[2], y0[2])
    e4 = f(t[3], y1[3]) - f(t[3], y0[3])
    return y_[3] + h/24 * (55*e4 - 59*e3 + 37*e2 - 9*e1)
    

def predictor(y,t, h, f):
    k1 = f(t[0], y[0])
    k2 = f(t[1], y[1])
    k3 = f(t[2], y[2]) 
    k4 = f(t[3], y[3]) # most recent
    return y[3] + h/24 * (55*k4 - 59*k3 + 37*k2 - 9*k1)

euler_predictor = lambda y,t,h,f : y+h*f(t,y)
euler_corrector = lambda y1,y0,t,h,f : y1 + h*(f(t,y1)-f(t,y0))

## Parallised version

In [ ]:
def iridc_ab(a,b,alpha, N, p, K, f):
    """Perform IDCp-FE
    Input: (a,b) endpoints; alpha ics; N #intervals; p order; K intervals; f vector field.
    Require: N divisible by K, with JK=N. M is #corrections. J groups of K intervals.
    Return: eta_sol
    """

    # Initialise, J intervals of size M 
    if not isinstance(N, int):
        raise TypeError('N must be integer')
    M = p-1
    if N % K !=0:
        raise Exception('K does not divide N')
    J = int(N/K)
    h = float(T)/N
    # number of equations in ODE (aka degree of freedom, dimension of space)
    #d = len(y0)
    S = np.zeros([M, M+1])

    for m in range(M):
        for i in range(M+1):
            c = lambda t,i : reduce(lambda x, y: x*y, 
                                [(t-k)/(i-k) for k in range(M) if k!=i])
            S[m,i] = quadrature(c, m, m+1, args=(i))[0] 

    Svec = S[M-1, :]
    # storing the final answer in yy
    eta_sol = np.zeros([N+1])
    # the time vector
    t = np.arange(0, T+h, h)
    t_ext = np.arange(0, T+h+M*h, h)
    # putting the initial condition in eta_sol
    eta_sol[0] = y0
    # loop over each group of intervals j
    for j in range(J):   
        print('Interval, ', j)
        # ----------------------------------------------------------------------
        # Phase 1: compute to the point every threads can start simultaneously
        eta_begin = np.zeros([M+1, 2*M])
        # predictor starts with last point in j-1 interval
        eta_begin[0, 0] = eta_sol[j*K]

        # predictor loop using forward Euler method
        for m in range(3):
            t[m+1] = (j*K+m+1) * h
            eta_begin[0, m+1] = euler_predictor(eta_begin[0, m], t[j*K+m], h, ff)
        for m in range(2*M-1):
            t[m+1] = (j*K+m+1) * h
            eta_begin[0, m+1] = predictor(eta_begin[0,m-3:m+1], t[j*K+m-3:j*K+m+1], h, ff)

        # corrector loops using Lagrange polynomials
        for l in range(1, M+1):
            eta_begin[l,:] = np.zeros([2*M])
            eta_begin[l, 0] = eta_sol[j*K]
            for m in range(3):
                eta_begin[l,m+1] = euler_corrector(eta_begin[l,m], eta_begin[l-1,m], t[j*K+m], h, ff)\
                    + h*sum([S[m, i]*ff(t[j*K+i], eta_begin[l-1, i]) for i in range(M+1)])
            for m in range(3:M):
                eta_begin[l, m+1] = corrector(eta_begin[l,m-3:m+1], eta_begin[l-1,m-3:m+1], t[j*K+m-3:j*K+m+1], h, ff)\
                    + h*sum([S[m, i]*ff(t[j*K+i], Ybegin[l-1, i]) for i in range(M+1)])
            for m in range(M, 2*M-l):
                eta_begin[l, m+1] = corrector(eta_begin[l,m-3:m+1], eta_begin[l-1,m-3:m+1], t[j*K+m-3:j*K+m+1], h, ff)\
                    + h*sum([S[M-1, i]*ff(t[j*K+m-M+i+1], eta_begin[l-1, m-M+i+1]) for i in range(M+1)])

        eta_pred = eta_begin[0, -1]
        eta_sol[j*K:j*K+M] = eta_begin[M, :M]
        # declare and fill up Y1corr and Y2corr for phase two
        eta0 = np.zeros([M, M+1])
        eta1 = np.zeros([M])
        for l in range(1, M+1):
            # 'lm' is for corrector 'l' (trying to save space for Y1corr&Y2corr
            lm = l - 1
            eta0[lm, :] = eta_begin[l-1, M-l:2*M-l+1]
            eta1[lm] = eta_begin[l, 2*M-l-1]

        # ----------------------------------------------------------------------
        # Phase 2: all threads can go simultaneously now
        for m in range(M-1, K):
            # predictor
            ts_pred = t_ext[j*K+M+m-3 : j*K+M+m+1]
            Ypred = predictor(Ypred, Tpred)


            # correctors
            for l in range(1, M+1):
                lm = l - 1
                Tvec = t_ext[j*K+m-l+1:j*K+m-l+1+M+1]
                Y2corr[:, lm] = corrector(Y2corr[:, lm], Y1corr[:, lm, :], Tvec)
            # update the stencil
            Y1corr[:, 0, 0:M] = Y1corr[:, 0, 1:M+1]
            Y1corr[:, 0, M] = Ypred
            for lm in range(1, M):
                Y1corr[:, lm, 0:M] = Y1corr[:, lm, 1:M+1]
                Y1corr[:, lm, M] = Y2corr[:, lm-1]
            # put the most corrected point in the final answer
            yy[:, j*K+m+1] = Y2corr[:, M-1]


    return t, eta_sol

In [ ]:
T = 10.0
y0 = np.array([1.0, 0])
p = 4  # RIDC(6,40)
M = p-1
K = 200
N = 1000
start = time.perf_counter()
tt, yy = RIDCsolver(func, T, y0, N, M, K)
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} second(s)')
print(tt[-1:])
print(yy[:, -1])

In [1]:
x = [1,2,3,4,5,6]
x[-3:]

[4, 5, 6]

## Test

In [46]:
def ridc_ab4(a,b,alpha, N, p, K, f):
    """Perform IDCp-FE
    Input: (a,b) endpoints; alpha ics; N #intervals; p order; K intervals; f vector field.
    Require: N divisible by K, with JK=N. M is #corrections. J groups of K intervals.
    Return: eta_sol
    """

    # Initialise, J intervals of size M 
    if not isinstance(N, int):
        raise TypeError('N must be integer')
    M = p-1
    if N % K !=0:
        raise Exception('K does not divide N')
    dt = (b-a)/N
    J = int(N/K)
    S = np.zeros([M+1,M+1])

    # M corrections, M intervals I, of size J 
    eta_sol = np.zeros(N+1)
    eta_sol[0]=alpha
    eta = np.zeros([K+1])
    eta1 = eta
    t = np.arange(0,b+h,h)

    # Precompute integration matrix
    for m in range(M):
        for i in range(M+1):
            c = lambda t,i : reduce(lambda x, y: x*y, 
                                [(t-k)/(i-k) for k in range(M) if k!=i])
            S[m,i] = quadrature(c, m, m+1, args=(i))[0] 


    # First interval, AB(4) prediction and FE correction
    eta[0], eta_sol[0] = alpha, alpha 
    for m in range(3):
        eta[m+1] = eta[m] + dt*f(t[m],eta[m])
    for m in range(3:K):
        eta[m+1] = predictor(t[m-3:m+1], eta[m-3:m+1], dt, f)

    for l in range(1,M+1):
            eta1[0] = eta[0]

            for m in range(3):
                term1 = dt*(f(t[m], eta1[m])-f(t[m], eta[m]))
                term2 = dt*np.sum([S[m,i] * f(t[i],eta[i]) for i in range(M+1)])
                eta1[m+1] = eta1[m] + term1 + term2
            for m in range(3,M):
                # Error equation, AB
                term1 = dt*np.sum([S[m,i] * f(t[i],eta[i]) for i in range(M+1)])
                eta1[m+1] = corrector(t[m-3:m+1], eta1[m-3:m+1], eta[m-3:m+1]) + term1
            for m in range(M,K):
                term1 = dt*np.sum([S[m,i] * f(t[i],eta[i]) for i in range(M+1)])
                eta1[m+1] = corrector(t[m-3:m+1], eta1[m-3:m+1], eta[m-3:m+1]) + term2
            if(l != M):
                eta = eta1

    eta_sol[1: K+1] = eta1[1:]
    
    # Rest of the intervals, AB stencil with most corrected
    for j in range(1,J):
        init=j*K
        eta_pred = eta_sol
        
        # Prediction Loop ADAM BASHFORTH, first 3 in FE
        eta[init:init+4] = [predictor(t[init - (4-i):init] + t[init+0:init + i],\
                                     eta_sol[init-(4-i):init+i] + eta[init+0:init+i], dt, f)\
                                     for i in range(0,4)]

        for m in range(4,K):
            eta[init+m] = predictor(t[init + m-4:init + m], eta[init+m-4:init+m], dt, f)

            # Correction Loops
            for l in range(1,M+1):
                eta[init+0] = eta_sol[j-1]
                for m in range(M):
                    # Error equation, Forward Euler
                    term1 = dt*(f(t[init + m],eta[l,j,m])-f(t[j,m],eta[l-1,j,m]))
                    term2 = dt*np.sum([S[m,i] * f(t[j,i],eta[l-1,j,i]) for i in range(M)])
                    eta[l,j,m+1] = eta[l,j,m] + term1 + term2
                for m in range(M,K):
                    term1 = dt*(f(t[j,m],eta[l,j,m])-f(t[j,m],eta[l-1,j,m]))
                    term2 = dt*np.sum([S[M-1,i] * f(t[j,m-M+i],eta[l-1,j,m-M+i]) for i in range(M)])
                    eta[l,j,m+1] =  eta[l,j,m] + term1 + term2
        

        eta_sol[j*K+1:(j+1)*K +1] = eta[M,j,1:]
            
    return eta_sol

In [47]:
ts = np.linspace(0, 1, 101)
# a, b , alpha, N, p>4, K, f
y_pred = ridc_ab4(a=0,b=1,alpha=1,N=100,p=5,K=2,f=dy_dt)
y_true = [y(t) for t in ts]
plt.plot(ts, y_true, ts, y_pred)
plt.legend(['y_true', 'y_pred'])

IndexError: index 3 is out of bounds for axis 1 with size 3